In [1]:
import requests
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

/Users/paesir/Desktop/git/llama-index/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
urls = [
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_japan_2024.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf"
]

# 각 파일 다운로드
for url in urls:
    filename = url.split("/")[-1]  # URL에서 파일명 추출
    response = requests.get(url)
    
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"{filename} 다운로드 완료")

ict_japan_2024.pdf 다운로드 완료
ict_usa_2024.pdf 다운로드 완료


In [3]:
us_docs = SimpleDirectoryReader(
    input_files=["ict_usa_2024.pdf"]
).load_data()
jp_docs = SimpleDirectoryReader(
    input_files=["ict_japan_2024.pdf"]
).load_data()

print('미국 시장동향 문서의 개수:', len(us_docs))
print('일본 시장동향 문서의 개수:', len(jp_docs))

미국 시장동향 문서의 개수: 29
일본 시장동향 문서의 개수: 30


In [4]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [ ]:
# API 키 설정
api_key = "your-api-key-here"  # 여기에 실제 API 키 입력
# 또는 환경 변수에서 가져오기
# api_key = os.environ.get("OPENAI_API_KEY")

In [6]:
# OpenAI LLM 초기화 (API 키 설정)
llm = OpenAI(
    api_key=api_key,
    model="gpt-4o-mini",
    temperature=0
)

# 미국과 일본 문서를 각각의 벡터 인덱스로 생성
# 문서를 임베딩 모델을 통해 벡터화하여 인덱스를 구축
us_index = VectorStoreIndex.from_documents(us_docs, embed_model=embed_model)
jp_index = VectorStoreIndex.from_documents(jp_docs, embed_model=embed_model)

# 생성된 벡터 인덱스를 로컬 스토리지에 저장
# 이후 재사용시 다시 생성할 필요 없이 저장된 인덱스를 불러올 수 있음
us_index.storage_context.persist(persist_dir="./storage/us")
jp_index.storage_context.persist(persist_dir="./storage/jp")

# 벡터 인덱스를 쿼리 엔진으로 변환
# similarity_top_k=5는 질문과 가장 유사한 상위 5개의 문서 청크를 검색하도록 설정
us_engine = us_index.as_query_engine(similarity_top_k=5, llm=llm)
jp_engine = jp_index.as_query_engine(similarity_top_k=5, llm=llm)

In [7]:
response = us_engine.query("이 문서의 요약본을 한글로 작성해줘")
print(response)

문서의 요약본은 다음과 같습니다:

이 문서는 정보통신산업진흥원에서 발행한 자료로, 2024년 3월 8일에 발행되었습니다. 문서에는 미국의 정보통신 산업에 대한 다양한 정보와 통계가 포함되어 있으며, 참고문헌과 사이트 목록도 제공됩니다. 문서의 내용은 상업적 목적이나 다른 용도로 무단 전재, 변경, 제3자 배포가 금지되어 있으며, 인용 시 출처 표기와 원본 변경 불가 등의 규칙을 준수해야 합니다.


In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=us_engine,
        metadata=ToolMetadata(
            name="usa_ict",
            description=(
                "미국의 ICT 시장동향 정보를 제공합니다. 미국 ICT와 관련된 질문은 해당 도구를 사용하세요. "
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=jp_engine,
        metadata=ToolMetadata(
            name="japan_ict",
            description=(
                "일본의 ICT 시장동향 정보를 제공합니다. 일본 ICT와 관련된 질문은 해당 도구를 사용하세요."
            ),
        ),
    ),
]

In [9]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage

# Agent 생성
agent = ReActAgent(
    tools=query_engine_tools,
    llm=llm,
    verbose=False,
    max_iterations=10
)

# 생성 후 프롬프트 확인
print("Agent 클래스:", type(agent))
print("사용 가능한 메서드:")
print([m for m in dir(agent) if not m.startswith('_')])

Agent 클래스: <class 'llama_index.core.agent.workflow.react_agent.ReActAgent'>
사용 가능한 메서드:
['add_step', 'aggregate_tool_results', 'call_tool', 'can_handoff_to', 'construct', 'copy', 'description', 'dict', 'finalize', 'formatter', 'from_orm', 'get_prompts', 'get_tools', 'handle_tool_call_results', 'init_run', 'initial_state', 'json', 'llm', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'name', 'output_cls', 'output_parser', 'parse_agent_output', 'parse_file', 'parse_obj', 'parse_raw', 'reasoning_key', 'run', 'run_agent_step', 'schema', 'schema_json', 'setup_agent', 'start_event_class', 'state_prompt', 'stop_event_class', 'streaming', 'structured_output_fn', 'system_prompt', 'take_step', 'tool_retriever', 'tools', 'update_forward_re

In [10]:
react_system_header_str = """\
당신은 질문에 답변하는 것부터 요약 제공, 기타 여러 유형의 분석까지 다양한 작업을 돕기 위해 설계되었습니다.

## 도구
당신은 다양한 도구에 접근할 수 있습니다. 현재 작업을 완료하기 위해 적절하다고 판단되는 순서로 도구를 사용하는 것은 당신의 책임입니다. 이를 위해 작업을 하위 작업으로 나누고, 각 하위 작업에 다른 도구를 사용할 필요가 있을 수 있습니다.

당신은 다음 도구들에 접근할 수 있습니다:
{tool_desc}

## 출력 형식
질문에 답변하기 위해 다음 형식을 사용하십시오.

###
Thought: I need to use a tool to help me answer the question.
Action: 도구 이름 (사용할 도구 중 하나인 {tool_names})
Action Input: 도구에 대한 입력을 JSON 형식으로 제공하십시오. 예: {{\"input\": \"hello world\", \"num_beams\": 5}}
###

항상 'Thought'로 시작하십시오.

'Action Input'에서는 올바른 JSON 형식을 사용하십시오. 이렇게 쓰지 마십시오: {{'input': 'hello world', 'num_beams': 5}}.

이 형식이 사용되면, 사용자는 다음 형식으로 응답할 것입니다:

###
Observation: 도구 응답
###

이 형식을 계속 반복하여 더 이상 도구를 사용하지 않고 질문에 답변할 수 있을 만큼 충분한 정보를 얻을 때까지 진행하십시오. 그 시점에서는 반드시 다음 두 가지 형식 중 하나로 응답해야 합니다:

###
Thought: I can answer without using any more tools.
Answer: [여기에 답변을 작성하세요]
###

###
Thought: I cannot answer the question with the provided tools.
Answer: 죄송합니다. 해당 질문에 답변할 수 없습니다.
###

## 추가 규칙
- 답변은 반드시 질문에 도달하기까지의 과정을 설명하는 순차적인 항목들로 구성되어야 합니다. 여기에는 이전 대화의 내용이 포함될 수 있습니다.
- 각 도구의 함수 서명을 반드시 준수해야 하며, 함수가 인수를 기대할 경우 인수를 생략하지 마십시오.
- 답변은 반드시 '한글'로 상세하게 작성되어야 합니다.
- 질문에 대한 답변만 작성하세요. 질문과 관계없는 검색을 수행하지 마십시오. 이는 매우 중요합니다.
- Please follow the thought-action-input format.
- 하나의 질문에 많은 검색어가 포함되어져 있는 것처럼 보인다면 검색어를 나누어서 순차적으로 검색하십시오. 더 좋은 답변을 얻을 수 있을 것입니다.
- 도구를 사용하여 답변할 수 있는 주제라면 반드시 도구를 사용하시기 바랍니다. 이는 매우 중요합니다.
- 당신이 도구 없이 답변하는 것은 도구의 주제와 완전히 다른 주제의 질문이 들어왔을 때 뿐입니다. 도구와 연관된 질문이라면 반드시 도구를 호출하십시오. 이는 매우 중요하며 당신이 지켜야 할 1순위의 우선사항입니다.

## 현재 대화
아래는 인간과 어시스턴트 메시지가 교차되어 있는 현재 대화 내용입니다.

"""

In [11]:
# 새 프롬프트 적용
react_system_prompt = PromptTemplate(react_system_header_str)
agent.update_prompts({"react_header": react_system_prompt})

# react_header 확인 
print(f"react_header: {react_system_prompt.template[:200]}...")

react_header: 당신은 질문에 답변하는 것부터 요약 제공, 기타 여러 유형의 분석까지 다양한 작업을 돕기 위해 설계되었습니다.

## 도구
당신은 다양한 도구에 접근할 수 있습니다. 현재 작업을 완료하기 위해 적절하다고 판단되는 순서로 도구를 사용하는 것은 당신의 책임입니다. 이를 위해 작업을 하위 작업으로 나누고, 각 하위 작업에 다른 도구를 사용할 필요가 있을 수 있습...


In [12]:
response = await agent.run("미국과 한국의 ICT 기관 협력 사례")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


미국과 한국의 ICT 기관 협력 사례로는 여러 가지가 있습니다. 두 나라는 정부 기관과 민간 기업 간의 파트너십을 통해 다양한 이니셔티브를 진행해왔습니다. 예를 들어, 한국의 과학기술정보통신부는 미국의 기관들과 ICT 프로젝트에 협력하고 있습니다. 또한, 여러 한국 기업들이 미국 시장에 진출하며 스타트업 지원 및 기술 협력을 강화하고 있습니다. 구체적인 사례로는 한국 기업들이 미국 기술 분야에 대한 투자와 혁신 및 연구 촉진을 위한 공동 이니셔티브가 있습니다.


In [13]:
response = await agent.run("미국과 일본의 ICT 주요 정책의 공통점과 차이점을 설명해줘.")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


미국과 일본의 ICT 주요 정책에는 몇 가지 공통점과 차이점이 있습니다.

**공통점:**
1. **기술 혁신 촉진**: 두 나라 모두 기술 혁신을 통해 국가의 경쟁력을 강화하려고 합니다. 미국은 6G 기술 개발을 위한 공공-민간 파트너십을 강조하고, 일본은 AI 모델 성능 향상을 위한 데이터 제공을 촉진하고 있습니다.

2. **민간 부문과의 협력**: 미국은 스펙트럼 자원의 효율적인 사용을 위해 민간 부문과 협력하고 있으며, 일본도 AI 개발자와의 협력을 통해 정부 데이터를 제공하고 있습니다.

**차이점:**
1. **정책 초점**: 미국의 정책은 주로 스펙트럼 관리와 6G 기술 개발에 중점을 두고 있는 반면, 일본의 정책은 AI 데이터 제공과 AI 모델 개발에 중점을 두고 있습니다.

2. **구체적인 실행 방안**: 미국은 스펙트럼 정책의 현대화와 민간 협력을 통한 자원 효율성을 강조하는 반면, 일본은 정부 데이터의 홍보와 AI 개발자를 위한 상담 창구 마련 등 구체적인 실행 방안을 제시하고 있습니다.

이러한 공통점과 차이점은 각국의 기술 발전 방향과 우선순위에 따라 다르게 나타납니다.
